In [1]:
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True

In [2]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
import pandas as pd
lines= pd.read_csv('/content/jeju_final.txt', names=['src', 'tar'], sep='\t')
len(lines)

466

In [4]:
lines.sample(10)

,src,tar
127,가레죽,삽
50,마농,마늘
244,감장돌다,맴돌다
323,먹엄직이 살암직이 시상,먹어볼만한 살아볼만한 세상
387,어떵허코 어떵허코 어떵허코 어떵허코,어떡하지 어떡하지 어떡하지 어떡하지
226,옴탕호다,오목하다
429,눈물 남쩌 눈물 남서,눈물 나네 눈물이 나
150,고냉이,고양이
183,감저,고구마
181,송키,채소


In [5]:
lines.tar = lines.tar.apply(lambda x : '\t '+ x + ' \n')
lines.sample(10)

,src,tar
123,작박,\t 작은바가지 \n
273,소도리호다,\t 소문내다 \n
256,귀태우다,\t 은밀히알려주다 \n
421,저 보름이,\t 저 바람이 \n
244,감장돌다,\t 맴돌다 \n
79,쪼께,\t 조끼 \n
417,휘이이,\t 휘이이 \n
65,보선,\t 버선 \n
145,고냥 독생이,\t 굴뚝새 \n
340,진아영 이우다,\t 진아영 입니다 \n


In [ ]:
# 글자 집합 구축
src_vocab=set()
for line in lines.src: # 1줄씩 읽음
    for char in line: # 1개의 글자씩 읽음
        src_vocab.add(char)

tar_vocab=set()
for line in lines.tar:
    for char in line:
        tar_vocab.add(char)

In [7]:
src_vocab_size = len(src_vocab)+1
tar_vocab_size = len(tar_vocab)+1
print(src_vocab_size)
print(tar_vocab_size)

499
484


In [9]:
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))
print(src_vocab[45:75])
print(tar_vocab[45:75])

['귀', '그', '근', '글', '금', '기', '김', '깍', '꺼', '껍', '께', '껭', '껴', '꼬', '꼭', '꼴', '꼽', '꽃', '꽈', '꽉', '꽝', '꾸', '꿀', '꿩', '꿰', '끄', '끈', '끌', '끼', '나']
['근', '글', '금', '긋', '기', '긷', '길', '김', '까', '깨', '꺼', '껏', '께', '꼬', '꼭', '꽁', '꽂', '꽃', '꽉', '꾸', '꿀', '끄', '끈', '끌', '끓', '끔', '끼', '나', '난', '날']


In [10]:
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(src_to_index)
print(tar_to_index)

{' ': 1, '.': 2, '3': 3, '4': 4, '5': 5, '?': 6, '가': 7, '각': 8, '간': 9, '갈': 10, '감': 11, '갓': 12, '강': 13, '개': 14, '갱': 15, '거': 16, '걱': 17, '건': 18, '걸': 19, '검': 20, '것': 21, '겅': 22, '게': 23, '겡': 24, '경': 25, '고': 26, '곡': 27, '곤': 28, '곧': 29, '골': 30, '곱': 31, '공': 32, '과': 33, '관': 34, '괭': 35, '괴': 36, '괸': 37, '교': 38, '구': 39, '국': 40, '군': 41, '굴': 42, '굶': 43, '굼': 44, '궤': 45, '귀': 46, '그': 47, '근': 48, '글': 49, '금': 50, '기': 51, '김': 52, '깍': 53, '꺼': 54, '껍': 55, '께': 56, '껭': 57, '껴': 58, '꼬': 59, '꼭': 60, '꼴': 61, '꼽': 62, '꽃': 63, '꽈': 64, '꽉': 65, '꽝': 66, '꾸': 67, '꿀': 68, '꿩': 69, '꿰': 70, '끄': 71, '끈': 72, '끌': 73, '끼': 74, '나': 75, '난': 76, '날': 77, '남': 78, '났': 79, '낭': 80, '내': 81, '냉': 82, '냐': 83, '냑': 84, '냥': 85, '넘': 86, '네': 87, '넹': 88, '녀': 89, '녁': 90, '년': 91, '녜': 92, '노': 93, '놀': 94, '놈': 95, '놋': 96, '농': 97, '놩': 98, '누': 99, '눈': 100, '뉠': 101, '느': 102, '는': 103, '늘': 104, '니': 105, '님': 106, '다': 107, '닥': 108, '단': 109, '달': 110, '담': 11

In [11]:
encoder_input = []
for line in lines.src: #입력 데이터에서 1줄씩 문장을 읽음
    temp_X = []
    for w in line: #각 줄에서 1개씩 글자를 읽음
      temp_X.append(src_to_index[w]) # 글자를 해당되는 정수로 변환
    encoder_input.append(temp_X)
print(encoder_input[:5])

[[479, 183, 230, 1], [481, 196, 1], [310, 230, 1], [324, 203, 1], [252, 224, 189, 1]]


In [12]:
decoder_input = []
for line in lines.tar:
    temp_X = []
    for w in line:
      temp_X.append(tar_to_index[w])
    decoder_input.append(temp_X)
print(decoder_input[:5])

[[1, 3, 3, 462, 305, 230, 398, 3, 2], [1, 3, 3, 462, 197, 99, 3, 2], [1, 3, 305, 230, 398, 3, 2], [1, 3, 3, 316, 197, 99, 3, 2], [1, 3, 3, 418, 87, 3, 2]]


In [15]:
decoder_target = []
for line in lines.tar:
    t=0
    temp_X = []
    for w in line:
      if t>0:
        temp_X.append(tar_to_index[w])
      t=t+1
    decoder_target.append(temp_X)
print(decoder_target[:5])

[[3, 3, 462, 305, 230, 398, 3, 2], [3, 3, 462, 197, 99, 3, 2], [3, 305, 230, 398, 3, 2], [3, 3, 316, 197, 99, 3, 2], [3, 3, 418, 87, 3, 2]]


In [16]:
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print(max_src_len)
print(max_tar_len)

37
44


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

In [ ]:
from tensorflow.keras.utils import to_categorical
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

In [22]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model

encoder_inputs = Input(shape=(None, src_vocab_size))
encoder_lstm = LSTM(units=256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
# encoder_outputs도 같이 리턴받기는 했지만 여기서는 필요없으므로 이 값은 버림.
encoder_states = [state_h, state_c]
# LSTM은 바닐라 RNN과는 달리 상태가 두 개. 바로 은닉 상태와 셀 상태.

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [ ]:
decoder_inputs = Input(shape=(None, tar_vocab_size))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)
# 디코더의 첫 상태를 인코더의 은닉 상태, 셀 상태로 합니다.
decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

In [24]:
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, epochs=50, validation_split=0.2)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 372 samples, validate on 94 samples
Epoch 1/50
372/372 [==============================] - 9s 25ms/sample - loss: 3.5953 - val_loss: 2.5762
Epoch 2/50
372/372 [==============================] - 1s 2ms/sample - loss: 1.0237 - val_loss: 2.4668
Epoch 3/50
372/372 [==============================] - 1s 2ms/sample - loss: 1.0023 - val_loss: 2.2461
Epoch 4/50
372/372 [==============================] - 1s 2ms/sample - loss: 0.9460 - val_loss: 2.5317
Epoch 5/50
372/372 [==============================] - 1s 2ms/sample - loss: 0.9457 - val_loss: 2.3743
Epoch 6/50
372/372 [==============================] - 1s 2ms/sample - loss: 0.9251 - val_loss: 2.2102
Epoch 7/50
372/372 [==============================] - 1s 2ms/sample - loss: 0.8978 - val_loss: 2.2760
Epoch 8/50
372/372 [==============================] - 1s 2ms/sample - loss: 0.9060 - val_loss: 2.3257
Epoch 9/50
372/372 [========================

In [ ]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)


In [ ]:
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
# 문장의 다음 단어를 예측하기 위해서 초기 상태를 이전 상태로 사용
decoder_states = [state_h, state_c]
# 이번에는 훈련 과정에서와 달리 은닉 상태와 셀 상태인 state_h와 state_c를 버리지 않음.
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [ ]:
index_to_src = dict(
    (i, char) for char, i in src_to_index.items())
index_to_tar = dict(
    (i, char) for char, i in tar_to_index.items())

In [ ]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)
    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, tar_to_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ""
    while not stop_condition: #stop_condition이 True가 될 때까지 루프 반복
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]
        decoded_sentence += sampled_char

        # <sos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_tar_len):
            stop_condition = True

        # 길이가 1인 타겟 시퀀스를 업데이트 합니다.
        target_seq = np.zeros((1, 1, tar_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.

        # 상태를 업데이트 합니다.
        states_value = [h, c]

    return decoded_sentence

In [31]:
import numpy as np
for seq_index in range(30): # 입력 문장의 인덱스
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', lines.src[seq_index])
    print('정답 문장:', lines.tar[seq_index][1:len(lines.tar[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1]) # '\n'을 빼고 출력

-----------------------------------
입력 문장: 하르방 
정답 문장:   할아버지 
번역기가 번역한 문장:   아지 
-----------------------------------
입력 문장: 할망 
정답 문장:   할머니 
번역기가 번역한 문장:   아지 
-----------------------------------
입력 문장: 아방 
정답 문장:  아버지 
번역기가 번역한 문장:   아지 
-----------------------------------
입력 문장: 어멍 
정답 문장:   어머니 
번역기가 번역한 문장:   아지 
-----------------------------------
입력 문장: 비바리 
정답 문장:   처녀 
번역기가 번역한 문장:   아지 
-----------------------------------
입력 문장: 괸당 
정답 문장:   친척 
번역기가 번역한 문장:   아지 
-----------------------------------
입력 문장: 걸바시 
정답 문장:   거지 
번역기가 번역한 문장:   아지 
-----------------------------------
입력 문장: 넹바리 
정답 문장:   시집간색시 
번역기가 번역한 문장:   아지 
-----------------------------------
입력 문장: 다슴아돌 
정답 문장:   의붓아들 
번역기가 번역한 문장:   아지 
-----------------------------------
입력 문장: 말젯놈 
정답 문장:   세번째자식 
번역기가 번역한 문장:   아지 
-----------------------------------
입력 문장: 소나이 
정답 문장:   사나이 
번역기가 번역한 문장:   아지 
-----------------------------------
입력 문장: 성님 
정답 문장:   형님 
번역기가 번역한 문장:   아지 
-------------------------------